## Compute the psf from the deepcoadds

- author : Sylvie Dagoret-Campagne
- creation date : 2025-05-29
- last update : 2025-05-29 

In [ ]:
import astroquery
print(astroquery.__version__)

https://stackoverflow.com/questions/59699193/how-to-obtain-2d-cutout-of-an-image-from-a-skycoord-position

In [ ]:
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.wcs import WCS
import os
import re

In [ ]:
# For Angle conversion
from astropy.coordinates import Angle
import astropy.units as u
from astropy.coordinates import SkyCoord
import pandas as pd
import numpy as np
from astropy.nddata import Cutout2D

In [ ]:
from astropy.visualization import ZScaleInterval, AsinhStretch, ImageNormalize

In [ ]:
from IPython.display import Image, display

In [ ]:
plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

## Configuration

### Target in ECDFS

- path to article : https://arxiv.org/pdf/1104.0931
- visual selection of the tiles : https://archive.stsci.edu/prepds/gems/browser.html
- path file download : https://archive.stsci.edu/pub/hlsp/gems/v_mk1/

In [ ]:
fig_table1 = "input_figs/table1_gemcandidates.png"
fig_table2 = "input_figs/table2_gemcandidates.png"

In [ ]:
Image(url= fig_table1,width=800)

In [ ]:
Image(url= fig_table2,width=500)

In [ ]:
#15422 44 03:32:38.21 –27:56:53.2 
ra1 = "03:32:38.21 hours"
dec1 = "-27:56:53.2 degrees"
tile1 = 44

#34244 94 03:32:06.45 –27:47:28.6 
ra2 = "03:32:06.45 hours"
dec2 = "-27:47:28.6 degrees"
tile2 = 94
# Je ne trouve pas cette tile ==> FindTileForCutoutGEM
tile2 = 32

#40173 35 03:33:19.45 –27:44:50.0 
ra3 = "03:33:19.45 hours"
dec3 = "-27:44:50.0 degrees"
tile3 = 35

#43242 45 03:31:55.35 –27:43:23.5 
ra4 = "03:31:55.35 hours"
dec4 = "-27:43:23.5 degrees"
tile4 = 45

#46446 47 03:31:35.94 –27:41:48.2 
ra5 = "03:31:35.94 hours"
dec5 = "-27:41:48.2 degrees"
tile5 = 47

#12589 03:31:24.89 −27:58:07.0
ra6 = "03:31:24.89 hours"
dec6 = "-27:58:07.0 degrees"
tile6 = 17

#43797 03:31:31.74 −27:43:00.8 
ra7 = "03:31:31.74 hours"
dec7 = "-27:43:00.8 degrees"
tile7 = 47

#28294 03:31:50.54 −27:50:28.4 
ra8 = "03:31:50.54 hours"
dec8 = "-27:50:28.4 degrees"
tile8 = 33

#36857 03:31:53.24 −27:46:18.9
ra9 = "03:31:53.24 hours"
dec9 = "-27:46:18.9 degrees"
tile9 = 38

#36714 03:32:59.78 −27:46:26.4 
ra10 = "03:32:59.78 hours"
dec10 = "-27:46:26.4 degrees"
tile10 = 37


In [ ]:
ra = Angle(ra10)
print(ra.degree)
dec = Angle(dec10)
print(dec.degree)

In [ ]:
lsstcomcam_targets = {}
# high rank
lsstcomcam_targets["ECDFS_G15422"] = {"field_name": "GEMS-15422", "ra": 53.159208333333325, "dec": -27.94811111111111,"tile":tile1}
lsstcomcam_targets["ECDFS_G34244"] = {"field_name": "GEMS-34244", "ra": 53.02687499999999 , "dec": -27.79127777777778,"tile":tile2}
lsstcomcam_targets["ECDFS_G40173"] = {"field_name": "GEMS-40173", "ra": 53.33104166666666 , "dec": -27.747222222222224,"tile":tile3}
lsstcomcam_targets["ECDFS_G43242"] = {"field_name": "GEMS-43242", "ra": 52.980624999999996 , "dec": -27.72319444444444,"tile":tile4}
lsstcomcam_targets["ECDFS_G46446"] = {"field_name": "GEMS-46446", "ra": 52.89975 , "dec": -27.696722222222224,"tile":tile5}

# low rank
lsstcomcam_targets["ECDFS_G12589"] = {"field_name": "GEMS-12589", "ra": 52.85370833333333, "dec": -27.96861111111111,"tile":tile6}
lsstcomcam_targets["ECDFS_G43797"] = {"field_name": "GEMS-43797", "ra": 52.88224999999999, "dec": -27.71688888888889,"tile":tile7}

lsstcomcam_targets["ECDFS_G28294"] = {"field_name": "GEMS-28294", "ra": 52.960583333333325 , "dec": -27.84122222222222,"tile":tile8}
lsstcomcam_targets["ECDFS_G6857"] = {"field_name": "GEMS-6857", "ra": 52.97183333333333 , "dec": -27.771916666666666,"tile":tile9}
lsstcomcam_targets["ECDFS_G36714"] = {"field_name": "GEMS-36714", "ra": 53.249083333333324, "dec": -27.773999999999997,"tile":tile10}


In [ ]:
df = pd.DataFrame(lsstcomcam_targets).T

In [ ]:
# candidates
key = "ECDFS_G15422"
#key = "ECDFS_G34244"
#key = "ECDFS_G40173"
#key= "ECDFS_G43242"
#key= "ECDFS_G46446"

# unknown
#key = "ECDFS_G12589"
#key = "ECDFS_G43797"
#key = "ECDFS_G28294"
#key = "ECDFS_G6857"
#key = "ECDFS_G36714"

the_target = lsstcomcam_targets[key]
target_ra = the_target["ra"]
target_dec = the_target["dec"]
target_name = the_target["field_name"]
tile_num = the_target["tile"]
coord = SkyCoord(ra=target_ra, dec=target_dec, unit=(u.deg, u.deg))

target_title = (
    the_target["field_name"] + f" (ra,dec) = ({target_ra:.2f},{target_dec:.2f}) "
)

### Access to remote files

- visual selection of the tiles : https://archive.stsci.edu/prepds/gems/browser.html
- path file download : https://archive.stsci.edu/pub/hlsp/gems/v_mk1/

### Config

- http://archive.stsci.edu/pub/hlsp/gems/v_mk1/h_gems_v35_mk1.fits
- url = f"http://archive.stsci.edu/pub/hlsp/gems/z_mk1/h_gems_z{tile_num}_mk1.fits"

In [ ]:
toppath_rubincutout = "../2025-05-24-SL-ECDFS-Fit/data"

In [ ]:
all_bands = ["u", "g", "r", "i", "z", "y"]
all_bands_colors = ["blue", "green", "red", "orange", "grey", "purple"]

### Choose the good index

In [ ]:
#all_gems_id_numbers = [15422,34244,40173,43242,46446, 12589,43797,28294,6857,36714]

In [ ]:
sl_number = int(re.findall("^ECDFS_G(.*)$",key)[0])
subpath = f"G{sl_number}"
path = os.path.join(toppath_rubincutout,subpath)

In [ ]:
fig,axs = plt.subplots(2,3,figsize=(12,8),layout="constrained")
all_cmaps = ["Blues","Greens","Reds","Oranges","Greys","Purples"]

ax = axs.flatten()

for ib,band in enumerate(all_bands):
    
    title = f"lsstcomcam psf band={band}"    
    suptitle = f"SL {sl_number}"

# --- Remplace par tes fichiers FITS ---
    file = f"cutout_psf_ECDFS_{subpath}_{band}.fits"


    fullfilename = os.path.join(path ,file)


    # --- Ouvrir les fichiers et leurs WCS ---
    hdu = fits.open(fullfilename)[0]

    data = hdu.data



    zscale = ZScaleInterval()
    vmin, vmax = zscale.get_limits(data)
  
    # --- Normalisation avec transformation asinh ---
    norm = ImageNormalize(vmin=vmin, vmax=vmax, stretch=AsinhStretch())

    # --- Afficher les images ---
    im = ax[ib].imshow(data, origin='lower', cmap=all_cmaps[ib], norm=norm)

# --- Ajouter des axes WCS ---
    ax[ib].set_xlabel('x')
    ax[ib].set_ylabel('y')
    ax[ib].set_title(title)

    ax[ib].set_aspect('equal', adjustable='datalim')



# --- Affichage final ---
plt.suptitle(suptitle + " PSF")
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import center_of_mass
from scipy.optimize import curve_fit
from astropy.io import fits
import pandas as pd

def gauss(x, A, mu, sigma):
    return A * np.exp(-0.5 * ((x - mu) / sigma)**2)

fig, axs = plt.subplots(2, 3, figsize=(12, 7), layout="constrained")
ax = axs.flatten()

# Tableau pour stocker les résultats
psf_results = []

for ib, band in enumerate(all_bands):
    
    title = f"lsstcomcam psf band={band}"    
    suptitle = f"SL {sl_number}"
    file = f"cutout_psf_ECDFS_{subpath}_{band}.fits"
    fullfilename = os.path.join(path, file)

    # Ouvrir FITS
    hdu = fits.open(fullfilename)[0]
    data = hdu.data

    # Trouver le centre
    y0, x0 = center_of_mass(data)

    # Calcul du rayon
    Y, X = np.indices(data.shape)
    r = np.sqrt((X - x0)**2 + (Y - y0)**2)
    r = r.astype(int)

    # Profil radial moyen
    tbin = np.bincount(r.ravel(), weights=data.ravel())
    nr = np.bincount(r.ravel())
    radialprofile = tbin / nr
    radii = np.arange(len(radialprofile))

    # Calcul sigma statistique
    flux_norm = radialprofile / np.sum(radialprofile)
    mu_stat = np.sum(radii * flux_norm)
    sigma_stat = np.sqrt(np.sum(flux_norm * (radii - mu_stat)**2))

    # Fit gaussienne
    try:
        popt, pcov = curve_fit(gauss, radii, radialprofile, p0=[np.max(radialprofile), mu_stat, sigma_stat])
        A_fit, mu_fit, sigma_fit = popt
        fwhm = 2.3548 * sigma_fit
    except RuntimeError:
        sigma_fit = np.nan
        fwhm = np.nan

    fwhm_arcsec = fwhm * 0.2  # si 1 pixel = 0.2 arcsec
    
    # Tracer
    ax[ib].plot(radii, radialprofile, color=all_bands_colors[ib], label='profil radial',lw=3)
    if not np.isnan(sigma_fit):
        ax[ib].plot(radii, gauss(radii, *popt), 'k--', label='fit gaussien')

    # Textbox
    

    stats_text = (f"$\\sigma_{{stat}}$ = {sigma_stat:.2f} px\n"
              f"$\\sigma_{{fit}}$ = {sigma_fit:.2f} px\n"
              f"FWHM = {fwhm:.2f} px\n"
              f"FWHM = {fwhm_arcsec:.2f}\"")

    
    ax[ib].text(0.95, 0.95, stats_text, transform=ax[ib].transAxes,
                verticalalignment='top', horizontalalignment='right',
                fontsize=10, bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

    ax[ib].set_xlabel('Radius (pixels)')
    ax[ib].set_ylabel('Average Flux')
    ax[ib].set_title(title)
    ax[ib].legend()

    # ➕ Ajouter les résultats à la liste
    psf_results.append({
    'band': band,
    'sigma_stat_px': round(sigma_stat, 2),
    'sigma_fit_px': round(sigma_fit, 2),
    'fwhm_px': round(fwhm, 2),
    'fwhm_arcsec': round(fwhm_arcsec, 2)
    })
    

plt.suptitle(suptitle + " PSF")
plt.show()

# 📊 Créer un DataFrame
psf_df = pd.DataFrame(psf_results)
psf_df = psf_df.round(2)

# 📝 Sauvegarder en CSV si souhaité :
# psf_df.to_csv("psf_statistics.csv", index=False)

# Afficher dans la console
print(psf_df)
